## Imports

In [1]:
import pandas as pd
import numpy as np
import os
import folium
import ast
import textwrap
import googlemaps
from folium.plugins import Fullscreen, MiniMap

## Data Read-in

In [2]:
df = pd.read_csv("IL Realtors_NAR April 1 Donations - Sheet2.csv")

In [3]:
%store -r google_maps_API_Key
gmaps_key = googlemaps.Client(key=google_maps_API_Key)

%store -r map_box_api_key

In [4]:
df.head(1)

,Candidate Name,SUM of Donations,Won?,Race?,County,Region,location_for_geocode
0,Brian Brubaker,"13,387.80",YES,Mayor of Round Lake,Lake,Collar counties,"Round Lake, Lake, IL"


In [5]:
# Define the geocode function
def geocode(add):
    g = gmaps_key.geocode(add)
    if g:
        lat = g[0]["geometry"]["location"]["lat"]
        lng = g[0]["geometry"]["location"]["lng"]
        return (lat, lng)
    else:
        return None

# Apply geocoding to the 'geo_address' column and store the results in 'geocoded' column
df['geocoded'] = df['location_for_geocode'].apply(geocode)

In [6]:
df['geocoded'] = df['geocoded'].astype(str)
df[['lat', 'lon']] = df['geocoded'].apply(lambda x: (None, None) if x == 'None' else x.strip('()').split(', ', 1)).apply(pd.Series)
df['lat'] = df['lat'].astype(float)
df['lon'] = df['lon'].astype(float)

In [7]:
df = df.dropna(subset='location_for_geocode')

In [8]:
df['SUM of Donations'] = df['SUM of Donations'].replace(',','',regex=True).astype(float)

In [9]:
df['Won?'] = df['Won?'].str.replace("YES","Win",regex=True).replace("NO","Lose",regex=True)

In [14]:
df['geocoded'].value_counts()

geocoded
(42.0568276, -87.6872172)                  3
(42.353355, -88.0934145)                   1
(41.7508391, -88.1535352)                  1
(38.9687219, -90.1934335)                  1
(42.2761341, -88.19202720000001)           1
(42.3702996, -87.90201859999999)           1
(41.4225316, -87.98588629999999)           1
(42.3579855, -88.0705319)                  1
(41.571701, -87.6944915)                   1
(40.5142026, -88.9906312)                  1
(40.94781580000001, -90.3712395)           1
(42.3800209, -88.2420315)                  1
(41.357254, -88.4211785)                   1
(38.7292147, -89.8831541)                  1
(42.34446639999999, -88.04174599999999)    1
(38.6818797, -90.1521665)                  1
(40.4768909, -88.9920118)                  1
(41.5244335, -88.0854555)                  1
(42.0883603, -87.98062650000001)           1
(41.1200325, -87.8611531)                  1
(42.3569661, -88.0767474)                  1
Name: count, dtype: int64

In [10]:
map_df = df

In [16]:
import pandas as pd
import folium
from folium.plugins import Fullscreen, MiniMap, MarkerCluster

# Function to create an HTML popup for each marker using candidate fields.
def create_popup(row):
    html = f"""
    <div class="popup-content">
        <strong>Candidate:</strong> {row['Candidate Name']}<br>
        <strong>Donations:</strong> ${float(row['SUM of Donations']):,.2f}<br>
        <strong>Outcome:</strong> {row['Won?']}<br>
        <strong>Race:</strong> {row['Race?']}<br>
        <strong>County:</strong> {row['County ']}<br>
        <strong>Region:</strong> {row['Region']}
    </div>
    """
    return html

# Create a color mapping based on Region.
unique_regions = map_df['Region'].unique()
colors = ['red', 'blue', 'green', 'purple', 'orange', 'darkred',
          'lightred', 'beige', 'darkblue', 'darkgreen', 'cadetblue',
          'darkpurple', 'white', 'pink', 'lightblue', 'lightgreen',
          'gray', 'black', 'lightgray']
region_color = {region: colors[i % len(colors)] for i, region in enumerate(unique_regions)}

# Initialize the map using one of the candidate's lat/lon values.
set_lat = float(map_df['lat'].iloc[6])
set_lon = float(map_df['lon'].iloc[6])
m = folium.Map(location=[set_lat, set_lon], zoom_start=7, scrollWheelZoom=False)

# Add custom Mapbox tile layer (replace YOUR_MAPBOX_API_KEY with your actual key).
folium.TileLayer(
    tiles=f'https://api.mapbox.com/styles/v1/mapbox/streets-v11/tiles/256/{{z}}/{{x}}/{{y}}@2x?access_token={map_box_api_key}',
    attr='Mapbox',
    name='Streets',
    overlay=True,
    control=False,
    show=False,
    min_zoom=1,
    max_zoom=20
).add_to(m)

# (Optional) Add custom CSS to style popups.
custom_css = """
<style>
    .popup-content {
        min-width: 250px;
        font-size: 14px;
        line-height: 1.4;
        color: #333;
        white-space: normal;
        word-wrap: break-word;
    }
    .leaflet-popup, .leaflet-popup-content-wrapper {
        background-color: #f9f9f9;
        border: 1px solid #bbb;
        border-radius: 5px;
        padding: 8px;
        box-shadow: 0 2px 6px rgba(0,0,0,0.1);
    }
    .leaflet-popup-tip {
        background: #f9f9f9;
    }
</style>
"""
m.get_root().html.add_child(folium.Element(custom_css))

# (Optional) Add a title to the map.
title_html = '''
    <h3 style="text-align:center; font-family:Arial, sans-serif; font-size:18px; color:#333; margin-top:10px;">
        <b>NAR's Illinois Political Donations</b>
    </h3>
'''
m.get_root().html.add_child(folium.Element(title_html))

# Add additional controls.
Fullscreen().add_to(m)
MiniMap(toggle_display=True).add_to(m)

# Create a MarkerCluster and add it to the map.
marker_cluster = MarkerCluster().add_to(m)

# Loop through the DataFrame rows to add markers to the MarkerCluster.
for i, row in map_df.iterrows():
    lat = float(row['lat'])
    lon = float(row['lon'])
    popup_html = create_popup(row)
    # Use the region color for the marker icon.
    color = region_color.get(row['Region'], 'blue')
    
    folium.Marker(
        location=[lat, lon],
        popup=folium.Popup(popup_html, max_width=300),
        icon=folium.Icon(color=color, icon='info-sign')
    ).add_to(marker_cluster)

# Create a legend for region colors.
legend_html = '''
     <div style="
     position: fixed; 
     bottom: 50px; left: 50px; width: 180px; 
     background-color: white;
     border:2px solid grey;
     z-index:9999;
     font-size:14px;
     padding: 10px;
     ">
     <strong>Legend (Region)</strong><br>
'''
for region, color in region_color.items():
    legend_html += f'<i style="background:{color};width:12px;height:12px;display:inline-block;margin-right:5px;"></i>{region}<br>'
legend_html += '</div>'
m.get_root().html.add_child(folium.Element(legend_html))


m

In [17]:
m.save("index.html")

In [18]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/NAR_IL_Donations_Map
